In [1]:
# Preparations
import math
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import numpy as np
from scipy import stats
from scipy.optimize import curve_fit
import seaborn as sns
from IPython.display import Latex
import warnings
from PrettyTable import PrettyTable
from functools import partial
from PrettyFigure import PrettyFigure
warnings.filterwarnings("ignore", module="matplotlib")
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
plt.rcParams['savefig.dpi'] = 75

# plt.rcParams['figure.autolayout'] = False
# plt.rcParams['figure.figsize'] = 10, 6
plt.rcParams['axes.labelsize'] = 18
plt.rcParams['axes.titlesize'] = 20
plt.rcParams['font.size'] = 16
plt.rcParams['lines.linewidth'] = 2.0
plt.rcParams['lines.markersize'] = 8
plt.rcParams['legend.fontsize'] = 14

plt.rcParams['text.usetex'] = True
plt.rcParams['text.latex.unicode'] = True
plt.rcParams['font.family'] = "STIX"
plt.rcParams['text.latex.preamble'] = "\\usepackage{subdepth}, \\usepackage{type1cm}"

results = {}

sns.set(color_codes=True)

def average(data):
    return 1 / len(data) * sum(data)

def error(data, average_of_data):
    s = sum([(x - average_of_data)**2 for x in data])
    return math.sqrt(s / (len(data) * (len(data) - 1)))

def std_deviation(error_of_average, length_of_dataset):
    return error_of_average * math.sqrt(length_of_dataset)

def average_with_weights(data, weights):
    d = data
    w = weights
    return (d * w**-2).sum() / (w**-2).sum()

def error_with_weights(weights):
    w = weights
    return 1 / math.sqrt((w**-2).sum())

def wavg(group, avg_name, weight_name):
    d = group[avg_name]
    w = group[weight_name]
    return (d * w**-2).sum() / (w**-2).sum()

def werr(group, weight_name):
    return 1 / math.sqrt((group[weight_name]**-2).sum())

In [2]:
# Constants
c0 = 299792458 # [m / s]
f1 = 1 # [m]
f2 = 4.994 # [m]
s1 = 0.110 # [m]
s2 = 4.873 # [m]
f1_s = 0.003 # [m]
f2_s = 0.003 # [m]
s2_s = 0.003 # [m]
AE = 0.4 # [g / m^3]
fmax = 1 # [g / m^3]
phi = AE / fmax
T = 296.15 # [K]
ps = 0.03 # [Pa]
p = 9625 # [mbar]

nn1 = 2.762
nn = nn1 * 1e-4 + 1
pn = 1013.25 # [mbar]
Tn = 288.15 # [K]
beta = 4.292e-8 # [1 / mbar]
gamma = 3.43e-2 # [(nm)^2 / mbar]
lambda0 = 632.8 #[nm]
pw = AE * ps

n = (nn - 1) * p * Tn / (pn * T) - (beta - gamma / lambda0**2) * pw + 1
ctheoretisch = c0 / n # [m / s]

# Arbeitsgrundlagen

## Lichtgeschwindigkeit in der Theorie

Die Lichtgeschwindigkeit im Vakuum wurde 1983 durch die Internationale Konferenz für Mass und Gewicht (ICPM) auf exakt

<center>
$c_0 = 299'792'458 \frac{m}{s}$
</center>
    
festgelegt.

Im Medium kann die Lichtgeschwindigkeit über den Brechungsindex $n$ bestimmt werden mit

\begin{equation}
c = \frac{c_0}{n}
\label{eq:c}
\end{equation}

Der Brechungsindex wiederum kann durch

\begin{equation}
n = (n_n - 1) \cdot \frac{pT_n}{p_nT} - \Big(\beta-\frac{\gamma}{\lambda_0^2}\Big)\cdot p_w -1
\label{eq:n}
\end{equation}

bestimmt werden. Hierbei gilt:

<center>
$n_n = 1.00$  
$T_n = 15°C$  
$p_n = 1013.25mbar$  
$\beta = 4.292\cdot10^{-8}mbar^{-1}$  
$\gamma = 3.43\cdot10^{-2}(nm)^2mbar^{-1}$  
$\lambda_0 \approx \lambda = 632.8 nm$  
$p_w = AE \cdot p_s = 0.4 \cdot 0.03 \cdot 10^3 mbar$
</center>

Somit errechnet sich die theoretische Lichtgeschwindigkeit im Zimmer als $c = 299'029'100\frac{m}{s}$.

## Messung der Lichtgeschwindigkeit nach Michelson

Beim Versuch von Michelson geht es drum den Lichtstrahl eines Lasers über verschiedene Spiegel, Linsen und Blenden zu schicken. Etwa in der mitte der totalen Distanz trifft der Strahl auf einen rotierenden Laser. Da das Licht auch nur endlich schnell ist, trifft das Licht auf dem Rückweg nicht an der selben Stelle auf dem rotierenden Spiegel auf wie auf dem Hinweg.
Somit kann über die zurückgelegte Distanz und die Verschiebung des Aufpralls auf dem Spiegel, sowie der bekannten Drehzahl die Lichtgeschwindigkeit errechnet werden.

![Versuchsaufbau nach Michelson. Ennommen aus der Aufgabenstellung](images/michelson.png)

Der Versuchsaufbau kann äquialent wie folgt dargestellt werden

![Äuquvaente Linsenkonfiguration nach Michelson. Ennommen aus der Aufgabenstellung](images/aequiv.png)

Der rotierende Spiegel wird zwischen dem ersten und dem zweiten Reflektieren des Strahls um den Winkel $\delta$ gedreht

\begin{equation}
\delta = \omega\Delta t = \omega \frac{2(s_2+f_2)}{c}
\label{eq:delta}
\end{equation}

Aufgrund des Reflexionsgesetzes welches besagt dass der Einfallswinkel $\alpha$ dem Ausfallswinkel $\alpha'$ entspricht

\begin{equation}
\alpha = \alpha'
\label{eq:refl}
\end{equation}

wird klar, dass der Drehspiegel eine Richtungsänderung von $2\delta$ zur Folge hat.

Des Weiteren gilt

\begin{equation}
2\delta = sin\Big(\frac{x}{f1}\Big) \approx \frac{x}{f1}
\label{eq:kleinwinkel}
\end{equation}

da die Kleinwinkeländerung sicher zutrifft.

Leicht lässt sich somit die Lichtgeschwindigkeit als

\begin{equation}
c = 4\omega\frac{(s_2+f_2)f_1}{x}
\label{eq:c_exp}
\end{equation}

bestimmen.

Da die Messungen noch offset-behaftet sind, wird ein Linearer Fit mit der Formel

\begin{equation}
x = \frac{1}{c}8\pi f\cdot f_1(s_2+f_2) + x_0
\label{eq:x_fit}
\end{equation}

Die Steigung der gefitteten Gerade entspricht also

\begin{equation}
m = \frac{1}{c}8\pi\cdot f_1(s_2+f_2)
\label{eq:m_fit}
\end{equation}

womit die Lichtgeschwindigkeit dann mit

\begin{equation}
c = \frac{1}{m}8\pi\cdot f_1(s_2+f_2)
\label{eq:c_fit}
\end{equation}

berechnet werden kann.

# Durchführung

Um die Messung durchführen zu können, wurde die bereits augebaute Apparatur so eingestellt dass der zurückgeworfene Lichtstrahl im Okular sichtbar war.
Dazu wurde erst der Drehspiegel (DS) so ausgerichtet dass er parallel zum Hohlspiegel (HS) ist.
Danach wurden der Endspiegel (ES) und der Umlenkspiegel (US) so justiert, dass zuerst der Strahl bei ES ankommt und dann so, dass der Strahl auch bis zu US zurückgeworfen wird.
Zuletzt wurde US noch so justiert dass der Strahl auch auf das Okular trifft.
Brennweiten waren glücklicherweise schon eingestellt und mussten so nicht mehr justiert werden.

Danach wurde die gesamte Apparatur vermessen um später mit den Werten rechnen zu könne. Die gemessenen Werte sind hier aufgeführt.

<center>
f1 = 1 ± 0.003 m  
f2 = 4.994 ± 0.003 m  
s1 = 0.110 ± 0.003 m  
s2 = 4.873 ± 0.003 m  
AE = 0.4 $\frac{g}{m^3}$  
T = 296.15 K  
ps = 0.03 Pa  
p = 9625 mbar
</center>

# Auswertung

In [3]:
#x = (1 / c) * 2 * omega * (s2 + fs) * (2 * f1) + x0

# Read Data
dfb = pd.read_csv('data/measurements.csv')
ax = None
dfa = dfb.loc[dfb['observer'] == 'Almar']
dfa.is_copy = False
dfa.loc[:,'x'] = dfa.loc[:,'x'] * 10e-5
dfn = dfb.loc[dfb['observer'] == 'Noah']
dfn.is_copy = False
dfn.loc[:,'x'] = dfn.loc[:,'x'] * 10e-5

slopea, intercepta, r, p, sema = stats.linregress(dfa['f'], dfa['x'])
slopen, interceptn, r, p, semn = stats.linregress(dfn['f'], dfn['x'])

ax = dfa.plot(kind='scatter', x='f', y='x', label='gemessene Distanz [Almar]')
plt.plot(dfa['f'], dfa['f'] * slopea + intercepta, label='linearer Fit der Distanz [Almar]', axes=ax)
plt.plot(dfn['f'], dfn['f'] * slopen + interceptn, label='linearer Fit der Distanz [Noah]', axes=ax, color='red')
plt.scatter(dfn['f'], dfn['x'], label='gemessene Distanz [Noah]', axes=ax, color='red')
plt.xlabel('f [Hz]')
plt.ylabel('x [m]')
plt.ylim([0.005, 0.007])
plt.legend(bbox_to_anchor=(0.02, 0.98), loc=2, borderaxespad=0.2)

plt.close()
figure = PrettyFigure(
    ax.figure,
    label='fig:lichtgeschwindigkeiten_distanz',
    caption='Gemessene Distanz x im Okular. Dazu einen linearen Fit um die Mittlere Distanz zu bestimmen.'
)

figure.show()

slope = np.array((slopea, slopen))
slope_s = np.array((sema, semn))

#x = (1 / c) * 2 * omega * (s2 + fs) * (2 * f1) + x0
c = (1 / (slope) * 2 * 2 * math.pi * (s2 + f2) * (2 * f1))

Es ergibt sich eine Mittlere Steigung von m = {{ '{0:.2E}'.format(np.mean(slope)) }} ± {{ '{0:.2E}'.format(np.mean(slope_s)) }} $\frac{m}{Hz}$. Löst man dies nach c auf, so erhält man c = {{ '{0:.2E}'.format(np.mean(c)) }} $\frac{m}{s}$
Wobei wir für Almars Messung eine Geschwindigkeit von $c_A$ = {{ '{0:.2E}'.format(c[0]) }} $\frac{m}{s}$ und für für Noahs Messung eine Geschwindigkeit von $c_N$ = {{ '{0:.2E}'.format(c[1]) }} $\frac{m}{s}$ erhalten.

# Fehlerrechnung

Mithilfe des Gaussschen Fehlerfortpflanzungsgesetzes können wir den Fehler für c berechnen.

Dafür finden wir erst alle Partialableitungen ersten Grades des Termes für c in der Gleichung \ref{eq:c_fit}.
Diese sind

<center>
    $\frac{\partial c}{\partial m}$ = $-\frac{1}{m^2}8\pi f_1 (s_2 + f_2)$  
    $\frac{\partial c}{\partial f1}$ = $\frac{1}{m}8\pi (s_2 + f_2)$  
    $\frac{\partial c}{\partial f_2}$ = $\frac{1}{m}8\pi f_1 (s_2 + 1)$  
    $\frac{\partial c}{\partial s_2}$ = $\frac{1}{m}8\pi f_1 (1 + f_2)$
</center>

Somit errechnet sich der Fehler dann als

\begin{equation}
s_{\overline{c}} = \sqrt{ \Big(  \frac{\partial\overline{c}}{\partial\overline{m}} \Big|_{\overline{c}} \cdot s_{\overline{m}}  \Big)^2 + \Big(  \frac{\partial\overline{c}}{\partial\overline{f_1}} \Big|_{\overline{c}} \cdot s_{\overline{f_1}}  \Big)^2  + \Big(  \frac{\partial\overline{c}}{\partial\overline{f_2}} \Big|_{\overline{c}} \cdot s_{\overline{f_2}}  \Big)^2  + \Big(  \frac{\partial\overline{c}}{\partial\overline{s_2}} \Big|_{\overline{c}} \cdot s_{\overline{s_2}}  \Big)^2  }
\label{eq:error_c}
\end{equation}

In [4]:
d0 = - 1 / slope**2 * 8 * math.pi * f1 * (s2 + f2) * slope_s
d1 = 1 / slope * 8 * math.pi * (s2 + f2) * f1_s
d2 = 1 / slope * 8 * math.pi * f1 * (s2 + 1) * f2_s
d3 = 1 / slope * 8 * math.pi * f1 * (1 + f2) * s2_s
sc = np.sqrt(d0**2 + d1**2 + d2**2 + d3**2)

c_wmean = sum(1 / sc**2 * c) / sum(1 / sc**2)

s_wmean = 1 / np.sqrt(sum(1 / sc**2))

Somit ergeben sich die systematischen Fehler

<center>
    $s_{\overline{A}}$ = {{'{0:.2E}'.format(np.mean(sc[0]))}} $\frac{m}{s}$  
    $s_{\overline{N}}$ = {{'{0:.2E}'.format(np.mean(sc[1]))}} $\frac{m}{s}$
</center>

Nun kann Mithilfe des gewichteten quadratischen Mittels

\begin{equation}
\overline{x} = \frac{\sum^N_{i=1}g_{\overline{x}_i}\overline{x}_i}{\sum^N_{i=1}g_{\overline{x}_i}}
\label{eq:c_mean}
\end{equation}

Die mittlere Lichtgeschwindigkeit unter Berücksichtigung des Fehlers ermittlt werden. Diese ergibt sich als

<center>
$\overline{c}$ = {{ '{0:.2E}'.format(c_wmean) }} $\frac{m}{s}$
</center>

Die mittlere Unsicherheit kann durch

\begin{equation}
s_{\overline{x}} = \frac{1}{\sqrt{\sum^N_{i=1}g_{\overline{x}_i}}}
\label{eq:s_mean}
\end{equation}

errechnet werden.

Somit ist $s_{\overline{c}}$ als

<center>
$s_{\overline{c}}$ = {{ '{0:.2E}'.format(s_wmean) }} $\frac{m}{s}$ 
</center>

bestimmt.

Die Lichtgeschwindigkeit ergibt sich aus der Messung unter Berücksichtigung des Fehlers als

<center>
c = $\overline{c}$ + $s_{\overline{c}}$ = {{ '{0:.2E}'.format(c_wmean) }} ± {{ '{0:.2E}'.format(s_wmean) }} $\frac{m}{s}$ 
</center>

# Resultate


Im folgenden sind die Resultate aus den Rechungen noch einmal tabellarisch gegenübergestellt.

In [5]:
# Errechnete Werte tabellarisch

labels = ['Lichtgeschwindigkeit nach ICPM',
          'Lichtgeschwindigkeit nach Gleichung 2',
          'Lichtgeschwindigkeit nach Messung Almar',
          'Lichtgeschwindigkeit nach Messung Noah',
          'Lichtgeschwindigkeit nach Messung Almar\nund Noah gewichtet gemittelt'
]
x = [
    c0,
    ctheoretisch,
    c[0],
    c[1],
    c_wmean
]

xerr = [
    0,
    0,
    sc[0],
    sc[1],
    s_wmean
]
xstr = list(map(('{0:.2E}').format, x))
xerrstr = list(map(('{0:.2E}').format, xerr))
xerrrelstr = list(map(('{0:.2f}').format, np.array(xerr) / np.array(x) * 100))
data = PrettyTable(
    
    list(zip(labels, xstr, xerrstr, xerrrelstr)),
    caption='Die verschieden errechneten Lichtgeschwindigkeiten mit ihren Fehlern.',
    entries_per_column=len(xstr),
    extra_header=['Herkunft', 'c [mps]', 's\_c [mps]', 's\_c relativ [%]'],
    significant_digits=2
)
data.show()

Herkunft,c [mps],s\_c [mps],s\_c relativ [\%]
Lichtgeschwindigkeit nach ICPM,3.00E+08,0.00E+00,0.00
Lichtgeschwindigkeit nach Gleichung 2,2.99E+08,0.00E+00,0.00
Lichtgeschwindigkeit nach Messung Almar,3.03E+08,2.30E+06,0.76
Lichtgeschwindigkeit nach Messung Noah,3.06E+08,1.64E+06,0.54
Lichtgeschwindigkeit nach Messung Almar und Noah gewichtet gemittelt,3.05E+08,1.34E+06,0.44


In [6]:
# Errechnete Werte grafisch

y = np.linspace(1, 9, 5)
plt.errorbar(x, y, xerr=xerr, fmt='o')
plt.yticks(y, labels, rotation='horizontal')
plt.xlim((295e6, 308e6))
plt.ylim((0, 10))
figure = PrettyFigure(
    plt.gcf(),
    label='fig:lichtgeschwindigkeiten',
    caption='Die gemessenen Lichtgeschwindigkeiten und Literaturwerte im Vergleich.'
)
plt.close()
figure.show()

Anhand des statistischen Fehlers der beiden Messreihen kann man erkennen dass Noah ein wenig genauer gemessen hat.
Der relative Fehler ist kleiner als 1%, was auf eine sehr genaue Messung hindeutet.
Trotzdem liegen alle Berechneten Werte signifikant über den Literaturwerten.
Dies kann natürlich damit zusammenhängen dass Linsen und Blenden nicht justiert wurden und die ganze Messapparatur aus Zeitgründen nicht kalibriert wurde.
Aus dieser Sicht sind die Messergbnisse eigentlich sehr zufriedenstellend!

# Anhang

## Messwerte

In [7]:
# Messwerte

data = PrettyTable(
    list(zip(dfa['x'], dfa['f'])),
    caption='Messwerte für x gemessen von Almar. Negative Werte für f kennzeichnen dass der Motor in die andere Richtung drehte.',
    entries_per_column=len(dfa['x'] / 2),
    extra_header=['x [m]', 'f [Hz]']
)
data.show()

data = PrettyTable(
    list(zip(dfn['x'], dfn['f'])),
    caption='Messwerte für x gemessen von Noah. Negative Werte für f kennzeichnen dass der Motor in die andere Richtung drehte.',
    entries_per_column=len(dfn['x'] / 2),
    extra_header=['x [m]', 'f [Hz]']
)
data.show()

x [m],f [Hz]
0.006,100
0.0061,200
0.00615,300
0.00625,400
0.006350000000000001,508
0.00645,601
0.006500000000000001,705
0.00657,804
0.0066500000000000005,904
0.00672,1002


x [m],f [Hz]
0.00585,-102
0.005770000000000001,-203
0.00568,-298
0.0056,-403
0.005520000000000001,-503
0.00543,-599
0.005350000000000001,-700
0.005260000000000001,-801
0.00519,-902
0.00512,-1001
